# Introduction to multiphysics modeling of sizing scenarios

- Utilisation de la simulation pour dériver des exigences 
- Exemple sur un actionneur de tuyere : obtention des efforts et vitesses maximales
- Utilisation de Modelica 
- Utilisation de Python : odeint
